In [2]:
using DifferentialEquations, FileIO, Plots, LinearAlgebra
using Printf, LaTeXStrings

 Let’s use the simpler heat equation
 $$u_t = u_{xx}$$
 as a model, and the boundary condition is a periodic condition
 $$u(1,t)=u(0,t)$$

In [3]:
"""
    diffper(n,xspan)

Construct 2nd-order differentiation matrices for functions with
periodic end conditions, using `n` unique nodes in the interval
`xspan`. Returns a vector of nodes and the matrices for the first
and second derivatives.
"""
function diffper(n,xspan)
    a,b = xspan
    h = (b-a)/n
    x = @. a + h*(0:n-1)   # nodes, omitting the repeated data

    # Construct Dx by diagonals, then correct the corners.
    dp = fill(0.5/h,n-1)        # superdiagonal
    dm = fill(-0.5/h,n-1)       # subdiagonal
    Dx = diagm(-1=>dm,1=>dp)
    Dx[1,n] = -1/(2*h)
    Dx[n,1] = 1/(2*h)

    # Construct Dxx by diagonals, then correct the corners.
    d0 =  fill(-2/h^2,n)        # main diagonal
    dp =  ones(n-1)/h^2         # superdiagonal and subdiagonal
    Dxx = diagm(-1=>dp,0=>d0,1=>dp)
    Dxx[1,n] = 1/(h^2)
    Dxx[n,1] = 1/(h^2)

    return x,Dx,Dxx
end

diffper

The PDE  is now approximated by the semidiscrete problem
$$
\frac{d \mathbf{u}(t)}{d t} = \mathbf{D}_{xx} \mathbf{u}(t),
$$
which is simply a linear, constant-coefficient system of ordinary differential equations. Given the initial values  obtained from , we have an initial-value problem that we already know how to solve!

In [4]:
m = 100
x,Dx,Dxx = diffper(m,[0,1])
u0 = @. exp(-60*(x-0.5)^2);

In [21]:
tfinal = 0.25
ODE = (u,p,t) -> Dxx*u;  
IVP = ODEProblem(ODE,u0,(0,tfinal))

sol = solve(IVP, RK4())
t = sol.t
u = sol.u

3590-element Vector{Vector{Float64}}:
 [3.059023205018258e-7, 5.540560400445003e-7, 9.915464915201364e-7, 1.7533189909197773e-6, 3.0633545018509363e-6, 5.288372581358964e-6, 9.020597359264982e-6, 1.5203270329463628e-5, 2.5317872214037864e-5, 4.16587136478562e-5  …  6.772873649085378e-5, 4.16587136478562e-5, 2.5317872214037864e-5, 1.5203270329463628e-5, 9.020597359264982e-6, 5.288372581358964e-6, 3.0633545018509363e-6, 1.7533189909197773e-6, 9.915464915201364e-7, 5.540560400445003e-7]
 [4.992477613125326e-7, 1.0836378306591033e-6, 1.3650161353569812e-6, 2.5049934324738582e-6, 4.262726073090216e-6, 7.248492546813916e-6, 1.2182968414086569e-5, 2.0239643982271528e-5, 3.323503521718117e-5, 5.3942634377913176e-5  …  8.653865140774416e-5, 5.3942634377913176e-5, 3.323503521718117e-5, 2.023964398227153e-5, 1.218296841408657e-5, 7.2484925468139165e-6, 4.262726073090216e-6, 2.5049934324738595e-6, 1.3650161353569804e-6, 1.0836378306591038e-6]
 [8.307723023859802e-7, 1.1382351542932214e-6, 1.669389

In [26]:
anim = @animate for j in 1:20:3000
    plot(x,u[j],label=@sprintf("t=%.4f",t[j]),
      xaxis=(L"x"), yaxis=(L"u(x,t)",[0,1]),dpi=100,
      title="Heat equation by rk4")
end
mp4(anim,"heatRK4.mp4")

┌ Info: Saved animation to d:\Github\julia-implement-numerical-ode-pde\One_Dim_Heat_Transfer_Equation\heatRK4.mp4
└ @ Plots C:\Users\zhufa\.julia\packages\Plots\Hxe7H\src\animation.jl:156


Plots.AnimatedGif("d:\\Github\\julia-implement-numerical-ode-pde\\One_Dim_Heat_Transfer_Equation\\heatRK4.mp4")

mp4 (generic function with 2 methods)